In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../src/")
sys.path.append("../")
import model
import dataset
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from aux.visualization import visualize_features_map_for_comparision
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Main

In [3]:
def main(exp, selected_layer, selected_filter, class_index=950):
    num_class = 30
    class_index = class_index
    print("=> class index: {}".format(class_index))
    # Modified here
    backbone = "vgg11"
    resume = "072200-22"
    
    color_map = "jet"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    ## Load modal from dict
    
    # model
    net = model.Network(backbone=backbone, num_classes=1000, 
                        selected_layer=selected_layer)
    net.to(device)
    
    # resume from model
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
    resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False) 
    
    summary(net, (1, 224, 224))
    
    ## Load Original Data 
    
    # Load data
    mean, std = (0.1307,), (0.3081,)
    reverse_mean = (-0.1307,)
    reverse_std = (1/0.3081,)
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = dataset.MNIST(root="/media/lincolnzjx/HardDisk/Datasets/", 
                              is_train=True, transform=train_transform)
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        ])         
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
    
    ## Create some need path
    
    save_dir = "../../saved/pack/"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    ab_path = os.path.join(save_dir, exp)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
    index2image
    
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])
    
    # Load image
    # ext may be different.
    optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
    valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    
    ## Obtain feature from conv layers.
    
    ori_activation_maps = net.get_activation_maps(original_image, 
                                                  selected_layer=selected_layer)[0]
    opt_activation_maps = net.get_activation_maps(opt_image, 
                                                  selected_layer=selected_layer)[0]
    
    images_cpu, optimized_data_cpu, ori_activation_maps_cpu, \
       opt_activation_maps_cpu, optimized_data_cpu_scale, \
       diff_activation_maps_cpu,\
       ori_activation_maps_cpu_n1e1, opt_activation_maps_cpu_n1e1, \
       diff_activation_maps_cpu_n1e1,\
       ori_activation_maps_cpu_n2e1, opt_activation_maps_cpu_n2e1, \
       diff_activation_maps_cpu_n2e1,\
       ori_activation_maps_cpu_n4e1, opt_activation_maps_cpu_n4e1,\
       diff_activation_maps_cpu_n4e1 = \
       preprocess_arrays(images_cpu, optimized_data,
                         ori_activation_maps, opt_activation_maps,
                         selected_filter, color_map=color_map)
    concated_imgs, concated_fms, concated_imgs_fms, out_not_resize_img = \
      concat_imgs(
        images_cpu, optimized_data_cpu,
        ori_activation_maps_cpu, opt_activation_maps_cpu,
        optimized_data_cpu_scale, diff_activation_maps_cpu,
        ori_activation_maps_cpu_n1e1, opt_activation_maps_cpu_n1e1,
        diff_activation_maps_cpu_n1e1,
        ori_activation_maps_cpu_n2e1, opt_activation_maps_cpu_n2e1,
        diff_activation_maps_cpu_n2e1,
        ori_activation_maps_cpu_n4e1, opt_activation_maps_cpu_n4e1,
        diff_activation_maps_cpu_n4e1)
    # imgs
    save_imgs_path = os.path.join("../../saved/pack/", exp, 
                                  "concat.png")
    concated_imgs.save(save_imgs_path)
    # fms 
    save_fms_path = os.path.join("../../saved/pack/", exp, 
                                  "concat_fms.png")
    concated_fms.save(save_fms_path)
    # imgs & fms
    save_imgs_fms_path = os.path.join("../../saved/pack/", exp, 
                                      "concat_imgs_fms.png")
    concated_imgs_fms.save(save_imgs_fms_path)
    # not_resize
    save_imgs_not_resize_path = os.path.join("../../saved/pack/", exp,
                                             "concat_imgs_fm_not_resize.png")
    out_not_resize_img.save(save_imgs_not_resize_path)
    
    
    save_dict = {
        "save_dir": "../../saved/pack/"+ exp + "/feature_map/",
        "index2image": index2image, 
        "save_name": "layer-{}-{}.pdf"
    }
    try: 
        os.mkdir(save_dict["save_dir"])
    except FileExistsError:
        print("Directory has been created {}".format(save_dict["save_dir"]))
        
    conv_output_indexes_dict = {0:0}
    layer_index = 0
    print(ori_activation_maps.shape)
    visualize_features_map_for_comparision(img_index=0, layer_index=layer_index, 
                                           features_map=[ori_activation_maps], 
                                           opt_feature_map=[opt_activation_maps], 
                                           cols=8, 
                                           conv_output_index_dict=conv_output_indexes_dict, 
                                           save_dict=save_dict, is_save=True,
                                           #plt_mode="img_scale", 
                                           plt_mode="single", 
                                           color_map = color_map,
                                           layer_name=selected_layer,
                                           selected_filter=selected_filter)

## PATH

In [4]:
excel_path = "../../src/exps.xlsx"
sheet = "baseline_mnist"
excepts = ["052148"]

In [5]:
exp_index = "x072800"

## RUN

In [6]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "Done": str,
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "Alpha": int,
                                           "Beta": int,
                                           "Gamma": int,
                                           "cuda": int,
                                           "class_index": int,
                                           "num_class": int,
                                           "re_size": int,
                                           "dataset": str,
                                           "backbone": str}).values

for data in excel:
    exp, done, selected_layer, selected_filter, alpha, beta, gamma, cuda, \
        class_index, num_class, re_size, dataset_name, backbone = data
    if exp != exp_index:
        continue
        
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN on ", cuda, exp, selected_layer, selected_filter, 
                  class_index)
            main(exp, selected_layer, 
                 selected_filter, class_index)

=> RUN on  5 072800 1 22 0
=> class index: 0
Only Keep 1th layers before.
Resume from model from exp: 072200 at epoch 22
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             640
              ReLU-2         [-1, 64, 224, 224]               0
             VGG11-3         [-1, 64, 224, 224]               0
Total params: 640
Trainable params: 640
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 73.50
Params size (MB): 0.00
Estimated Total Size (MB): 73.69
----------------------------------------------------------------
Len of new dataset is :30
../../saved/pack/072800
(30, 224, 224, 3)


IndexError: tuple index out of range

In [ ]:
   mean, std = (0.1307,), (0.3081,)